# Install


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

import pandas as pd
from dotenv import load_dotenv

import utils
import model

load_dotenv("../user.env")

True

In [3]:
# def to_snake_case_with_dots(column_name):
#     result = column_name[0].lower()
#     for char in column_name[1:]:
#         if char.isupper():
#             result += "_" + char.lower()
#         elif char == ".":
#             result += char
#         else:
#             result += char
#     return result


# def clean_column_text(columns):
#     return [
#         keep_text_right_of_dot(to_snake_case_with_dots(column)) for column in columns
#     ]


# def keep_text_right_of_dot(column):
#     if "." not in column:
#         return column

#     result = column.split(".")[-1]
#     return result

# Mocked data


In [4]:
def get_mocked_data():
    def get_requisition():
        requisition = {
            "id": "738d03c6-96fe-4b9e-a10c-b5befb1d02c8",
            "created": "2023-12-25T09:33:37.022127Z",
            "redirect": "https://gocardless.com",
            "status": "LN",
            "institution_id": "SANTANDER_GB_ABBYGB2L",
            "agreement": "71321e76-07d3-4438-ad18-d600ce91438c",
            "reference": "073127d2-2018-4a4a-b47e-1170d8616228",
            "accounts": [
                "590300bd-3daf-4d5e-9274-7a3782261f7e",
                "d2ff77d0-6c80-4580-95a5-e3e87a098db9",
                "e9e5f8b9-da61-49ce-bdae-56546ce4a1c9",
            ],
            "link": "https://ob.nordigen.com/psd2/start/738d03c6-96fe-4b9e-a10c-b5befb1d02c8/SANTANDER_GB_ABBYGB2L",
            "ssn": None,
            "account_selection": False,
            "redirect_immediate": False,
        }
        return requisition

    def get_metadata():
        joint_account_metadata = {
            "id": "590300bd-3daf-4d5e-9274-7a3782261f7e",
            "created": "2023-03-13T17:58:53.898557Z",
            "last_accessed": "2023-12-30T14:42:46.907297Z",
            "iban": "GB52ABBY09012921263671",
            "institution_id": "SANTANDER_GB_ABBYGB2L",
            "status": "READY",
            "owner_name": "",
        }

        credit_card_metadata = {
            "id": "d2ff77d0-6c80-4580-95a5-e3e87a098db9",
            "created": "2023-03-13T17:58:53.845197Z",
            "last_accessed": "2023-12-30T14:40:20.696724Z",
            "iban": "**4572",
            "institution_id": "SANTANDER_GB_ABBYGB2L",
            "status": "READY",
            "owner_name": "",
        }

        single_account_metadata = {
            "id": "e9e5f8b9-da61-49ce-bdae-56546ce4a1c9",
            "created": "2023-03-13T17:58:53.874184Z",
            "last_accessed": "2023-12-30T14:53:28.379568Z",
            "iban": "GB31ABBY09012916688559",
            "institution_id": "SANTANDER_GB_ABBYGB2L",
            "status": "READY",
            "owner_name": "",
        }

        account_metadata = [
            joint_account_metadata,
            credit_card_metadata,
            single_account_metadata,
        ]

        return account_metadata

    def get_details():
        joint_account_details = {
            "account": {
                "resourceId": "cbc85e1c-cdbc-4e3a-9918-03b74d79e719",
                "bban": "09012921263671",
                "currency": "GBP",
                "name": "joint account",
                "cashAccountType": "CACC",
            }
        }
        credit_card_details = {
            "account": {
                "resourceId": "c4e9a964-b676-43c1-81f9-5479f8894604",
                "currency": "GBP",
                "cashAccountType": "CARD",
                "maskedPan": "**4572",
                "details": "STAFF ALL IN ONE CREDIT CARD",
            }
        }

        single_account_details = {
            "account": {
                "resourceId": "a2b51663-5cf3-43d6-866e-65e5c63542bc",
                "bban": "09012916688559",
                "currency": "GBP",
                "name": "single account",
                "cashAccountType": "CACC",
            }
        }

        account_details = [
            joint_account_details,
            credit_card_details,
            single_account_details,
        ]
        return account_details

    def get_balances():
        joint_account_balances = {
            "balances": [
                {
                    "balanceAmount": {"amount": "3700.80", "currency": "GBP"},
                    "balanceType": "forwardAvailable",
                    "referenceDate": "2023-12-30",
                },
                {
                    "balanceAmount": {"amount": "-1195.39", "currency": "GBP"},
                    "balanceType": "openingCleared",
                    "referenceDate": "2023-12-30",
                },
                {
                    "balanceAmount": {"amount": "-710.84", "currency": "GBP"},
                    "balanceType": "previouslyClosedBooked",
                    "referenceDate": "2023-12-26",
                },
            ]
        }

        credit_card_balances = {
            "balances": [
                {
                    "balanceAmount": {"amount": "3700.80", "currency": "GBP"},
                    "balanceType": "forwardAvailable",
                    "referenceDate": "2023-12-30",
                },
                {
                    "balanceAmount": {"amount": "-1195.39", "currency": "GBP"},
                    "balanceType": "openingCleared",
                    "referenceDate": "2023-12-30",
                },
                {
                    "balanceAmount": {"amount": "-710.84", "currency": "GBP"},
                    "balanceType": "previouslyClosedBooked",
                    "referenceDate": "2023-12-26",
                },
            ]
        }

        single_account_balances = {
            "balances": [
                {
                    "balanceAmount": {"amount": "1023.14", "currency": "GBP"},
                    "balanceType": "interimAvailable",
                    "referenceDate": "2023-12-30",
                },
                {
                    "balanceAmount": {"amount": "1023.14", "currency": "GBP"},
                    "balanceType": "interimBooked",
                    "referenceDate": "2023-12-30",
                },
            ]
        }

        account_balances = [
            joint_account_balances,
            credit_card_balances,
            single_account_balances,
        ]
        return account_balances

    requisition = get_requisition()
    account_metadata = get_metadata()
    account_details = get_details()
    account_balances = get_balances()
    transactions_raw = utils.read_json("../data/output.json").get("data")

    return requisition, transactions_raw, account_details

# Process data from API (mocked)


In [5]:
def parse_account_dict(account, id):
    def parse_credit_card(account: dict, id: str):
        return {
            "account_id": id,
            "account_name": account.get("details"),
            "account_type": account.get("cashAccountType"),
            "currency": account.get("currency"),
            "masked_pan": account.get("maskedPan"),
        }

    def parse_current_account_dict(account, id):
        return {
            "account_id": id,
            "account_name": account.get("name"),
            "account_type": account.get("cashAccountType"),
            "currency": account.get("currency"),
        }

    account_type = account.get("cashAccountType")

    if account_type == "CARD":
        return parse_credit_card(account, id)
    elif account_type == "CACC":
        return parse_current_account_dict(account, id)
    else:
        raise NotImplementedError(f"unhandled account_type={account_type}")


def extract_transactions_and_balance_from_response(account_id: str, transactions_raw):
    def combine_pending_and_booked_transactions(transactions: list[dict]):
        booked_df = pd.json_normalize(transactions.get("booked"))
        if booked_df is not None:
            booked_df["status"] = "booked"
        pending_df = pd.json_normalize(transactions.get("pending"))
        if pending_df is not None:
            pending_df["status"] = "pending"

        df = pd.concat([booked_df, pending_df])
        transactions = df.to_dict(orient="records")
        return transactions

    def find_account_transactions():
        # loop through looking for account id
        result = None
        finished = False

        for acc in transactions_raw:
            acc_id = acc[0]
            if acc_id == account_id:
                result = acc
        finished = True

        if finished and result is None:
            Exception(f"could not find account_id {account_id} in account details data")

        return result

    def extract_transactions(result):
        acc_transactions = result[1][0].get("transactions")
        return acc_transactions

    def extract_balances(result):
        acc_balances = result[1][1].get("balances")
        return acc_balances

    def add_account_id_to_transactions(transactions: list[dict], account_id: str):
        result = []

        for transaction in transactions:
            updated_dict = transaction
            updated_dict["account_id"] = account_id
            result.append(updated_dict)

        return result

    # parse
    result = find_account_transactions()
    acc_transactions = extract_transactions(result)
    acc_balances = extract_balances(result)

    # post process
    acc_transactions = combine_pending_and_booked_transactions(acc_transactions)
    acc_transactions = add_account_id_to_transactions(
        transactions=acc_transactions, account_id=account_id
    )

    return acc_transactions, acc_balances


def populate_user_accounts_data(requisition, transactions_raw, account_details):
    # Populate accounts
    accounts = [
        parse_account_dict(acc.get("account"), id=requisition.get("accounts")[idx])
        for idx, acc in enumerate(account_details)
    ]

    for account_id in requisition.get("accounts"):
        acc_position = requisition.get("accounts").index(account_id)
        account_name = accounts[acc_position].get("account_name")
        print(
            f"account_position={acc_position}, account_name={account_name}, account_id={account_id}"
        )

        transactions, balances = extract_transactions_and_balance_from_response(
            account_id, transactions_raw=transactions_raw
        )

        accounts[acc_position]["transactions"] = transactions
        accounts[acc_position]["balances"] = balances

    return accounts

In [6]:
requisition, transactions_raw, account_details = get_mocked_data()

In [7]:
user_accounts_data = populate_user_accounts_data(
    requisition=requisition,
    transactions_raw=transactions_raw,
    account_details=account_details,
)

account_position=0, account_name=joint account, account_id=590300bd-3daf-4d5e-9274-7a3782261f7e
account_position=1, account_name=STAFF ALL IN ONE CREDIT CARD, account_id=d2ff77d0-6c80-4580-95a5-e3e87a098db9
account_position=2, account_name=single account, account_id=e9e5f8b9-da61-49ce-bdae-56546ce4a1c9


# Parsed data to domain model

accounts


In [9]:
def get_account_data(account_data):
    result_dict = {}
    balances: list[dict] = account_data.get("balances")
    data_list = utils.flatten_and_remove_duplicates_from_dictionary(balances)
    for entry in data_list:
        balance_type = entry.get("balanceType")
        amount = entry.get("amount")

        if balance_type and amount:
            key = f"{balance_type}Balance"
            result_dict[key] = amount

    result_dict["currency"] = data_list[0]["currency"]
    result_dict["referenceDate"] = data_list[0]["referenceDate"]
    result_dict = utils.clean_column_names(result_dict)

    acc_detail = {
        "account_id": account_data.get("account_id"),
        "account_name": account_data.get("account_name"),
        "account_type": account_data.get("account_type"),
        "currency": account_data.get("currency"),
    }
    acc_detail.update(result_dict)

    return acc_detail


# clean
def transactions_dict_to_model(account_data: dict, account_id: str):
    transactions = utils.clean_column_names(account_data.get("transactions"))
    transactions = [model.Transaction.from_dict(d) for d in transactions]
    filtered_transactions = [t for t in transactions if t.account_id == account_id]
    return transactions

In [10]:
def get_user_selection(selected_account: int = 0):
    account_ids = [d.get("account_id") for d in user_accounts_data]
    if selected_account >= len(account_ids):
        raise ValueError("invalid entry")
    account_id = account_ids[selected_account]
    account_position = account_ids.index(account_id)
    return account_position


account_position = get_user_selection()

In [11]:
# user selects account
user_accounts = []
for i in range(0, len(user_accounts_data)):
    try:
        account_data = user_accounts_data[i]
        account_type = account_data.get("account_type")
        # initialise account
        if account_type == "CACC":
            account = model.BankAccount.from_dict(get_account_data(account_data))
        elif account_type == "CARD":
            account = model.CreditCard.from_dict(get_account_data(account_data))

        # initialise transactions
        transactions = transactions_dict_to_model(account_data, account.account_id)
        account.add_transactions(transactions=transactions)
        user_accounts.append(account)
    except Exception as e:
        print(f"failed to process account {i}")
        raise e

In [12]:
user_accounts

[Account='joint account' AccountType=CACC Id:=590300bd-3daf-4d5e-9274-7a3782261f7e InterimBalance=18544.39 OverdraftLimit=None,
 Account='STAFF ALL IN ONE CREDIT CARD' AccountType=CARD Id:=d2ff77d0-6c80-4580-95a5-e3e87a098db9 InterimBalance=None CreditLimit=4896.1900000000005,
 Account='single account' AccountType=CACC Id:=e9e5f8b9-da61-49ce-bdae-56546ce4a1c9 InterimBalance=1023.14 OverdraftLimit=None]

In [17]:
user = model.User(user_name="Matthew")

In [18]:
user = model.User(user_name="Matthew")
_ = [user.add_account(account) for account in user_accounts]

In [24]:
transactions = user.get_transactions()
transactions_by_amount = user.get_transactions_by_amount(100, 200)
transactions_by_amount = user.get_transactions_by_amount(400, 500)

In [25]:
from collections import OrderedDict

def find_common_transactions():
    ...

def find_common_dicts(*lists):
    def dict_to_ordered_tuple(d):
        return tuple(OrderedDict(sorted(d.items())).items())

    sets = [set(dict_to_ordered_tuple(d) for d in lst) for lst in lists]

    common_set = set.intersection(*sets)

    result = [dict(t) for t in common_set]
    return result

In [26]:
queries = [user.get_transactions(), user.get_transactions_by_amount(-100, -10)]
dicts = [[d.to_dict() for d in query] for query in queries]
common_dicts = find_common_dicts(*dicts)

In [27]:
query_response = [model.Transaction(**t) for t in common_dicts]

In [28]:
def transactions_to_dataframe(transactions: list[model.Transaction]):
    return pd.DataFrame(query_response).dropna(axis=1)

In [29]:
transactions_to_dataframe(query_response)

,booking_date,booking_date_time,remittance_information_unstructured,proprietary_bank_transaction_code,amount,status,transaction_id,internal_transaction_id,account_id,account_type,account_name,currency
0,2023-11-26,2023-11-26,CASH WITHDRAWAL AT BARCLAYS BANK PLC ATM ASDA ...,DEBIT CARD CASH WITHDRAWAL,-20.00,booked,66a188e2-1aac-310f-8d02-e5d3ddf9609a,aafc4ba5283b36f2b77dd83aa549ee2b,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
1,2023-11-08,2023-11-08,CARD PAYMENT TO APPLE.COM/BILL ON 08-11-2023,OTT DEBIT,-46.99,booked,ee3f7171-9dff-3b4f-88cb-626e79bf152c,c865df126132dcaed530fe6a5cee7562,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
2,2023-11-29,2023-11-29,DIRECT DEBIT PAYMENT TO AVIVA RCPTS ACCOUN REF...,EXTERNAL DIRECT DEBIT,-44.90,booked,b56da985-6adf-343d-88ec-cce4fc5c251f,3a11dd16667a10ab0e78e64fb6009b99,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
3,2023-12-28,2023-12-28,DIRECT DEBIT PAYMENT TO WINTERFLOOD SEC REF BK...,EXTERNAL DIRECT DEBIT,-20.00,booked,31257ccd-583a-3a11-a90b-ec9aebd62a48,6ed0227c07abf98e103371c2ed0aa49e,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
4,2023-10-30,2023-10-30,DIRECT DEBIT PAYMENT TO UNITE THE UNION REF 20...,EXTERNAL DIRECT DEBIT,-16.25,booked,7b5b5f93-0702-331d-a055-7d37c648b0ac,01a503309abab9f036965727e1eae2d3,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...
82,2023-12-27,2023-12-27,DIRECT DEBIT PAYMENT TO THAMES WATER REF 90007...,EXTERNAL DIRECT DEBIT,-50.00,booked,d5f73163-ea72-3a4d-b8ba-73c3ddae524c,413983ab6e73e63df466456d972308e1,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
83,2023-10-16,2023-10-16,DIRECT DEBIT PAYMENT TO IFL PET INSURANCE REF ...,EXTERNAL DIRECT DEBIT,-16.98,booked,666ce7ae-3f38-3c43-9c29-47164d1520bc,4dbedf0bfd91adcaa28b59e3f7999f83,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
84,2023-12-24,2023-12-24,CARD PAYMENT TO PIZZA HUT ONLINE WEB ON 23-12-...,OTT DEBIT,-41.27,booked,84093568-019f-33e0-9610-5c1374d52aa5,b064327abf49f98e5165ebe298f1f203,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
85,2023-12-18,2023-12-18,BILL PAYMENT VIA FASTER PAYMENT TO LUSH CLEANS...,BANK TRANSFER DEBIT,-54.00,booked,b6b5fcf2-5fef-3312-a345-7d08fcaf908a,d9052c9e759e4ff0fddacd69183df68c,590300bd-3daf-4d5e-9274-7a3782261f7e,CACC,joint account,GBP
